In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [2]:
tokenizer_geo = AutoTokenizer.from_pretrained('ZurabDz/albert-geo')
tokenizer_mix = AutoTokenizer.from_pretrained('/home/penguin/GeorgianWritingWizard/language_engine/examples/ff_tok')

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

In [5]:
tokenizer_geo.save_pretrained('geo')

('geo/tokenizer_config.json',
 'geo/special_tokens_map.json',
 'geo/tokenizer.json')

In [3]:
tokenizer_geo('გამარჯობა როგორ ხარ?')

{'input_ids': [6361, 4286, 4277, 12990, 4], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [4]:
tokenizer_mix('გამარჯობა როგორ ხარ?')

{'input_ids': [2, 8, 1, 8, 1, 8, 1, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [2]:
tokenizer = AutoTokenizer.from_pretrained('ZurabDz/albert-geo')
dataset = load_dataset('ZurabDz/geo_small_corpus')

Using custom data configuration ZurabDz--geo_small_corpus-297bec1b2e1a3b51
Found cached dataset parquet (/home/penguin/.cache/huggingface/datasets/ZurabDz___parquet/ZurabDz--geo_small_corpus-297bec1b2e1a3b51/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7375931
    })
})

In [4]:
raw_dataset = dataset['train']

In [8]:
tokenizer.model_max_length = 1e30

def filtering_rule(examples):
    tokenized = tokenizer(examples['text'])["input_ids"]
    return [len(t) < 0.3 * len(e) for t, e in zip(tokenized, examples['text'])]

In [9]:
map_setup = dict(
    batched=True,
    batch_size=1024,
    num_proc=8
        # load_from_cache_file=False,
        # keep_in_memory=False,
)

In [10]:
print(f"Size of dataset before trash removal: {len(raw_dataset)}.")
raw_dataset = raw_dataset.filter(
    filtering_rule,
    desc="Filter sentences that cannot be tokenized well.",
    **map_setup,
    # keep_in_memory=True,  # can run out of mem even on the 750GB node?
)
print(f"Size of filtered dataset: {len(raw_dataset)}.")

Size of dataset before trash removal: 7375931.
              

Filter sentences that cannot be tokenized well. #6:   0%|          | 0/901 [00:00<?, ?ba/s]

Filter sentences that cannot be tokenized well. #2:   0%|          | 0/901 [00:00<?, ?ba/s]

Filter sentences that cannot be tokenized well. #7:   0%|          | 0/901 [00:00<?, ?ba/s]

Filter sentences that cannot be tokenized well. #4:   0%|          | 0/901 [00:00<?, ?ba/s]

Filter sentences that cannot be tokenized well. #3:   0%|          | 0/901 [00:00<?, ?ba/s]

Filter sentences that cannot be tokenized well. #5:   0%|          | 0/901 [00:00<?, ?ba/s]

Filter sentences that cannot be tokenized well. #1:   0%|          | 0/901 [00:00<?, ?ba/s]

Filter sentences that cannot be tokenized well. #0:   0%|          | 0/901 [00:00<?, ?ba/s]

Size of filtered dataset: 6185382.


In [14]:
raw_dataset.save_to_disk('filtered_data')

Flattening the indices:   0%|          | 0/6186 [00:00<?, ?ba/s]

Saving the dataset (0/8 shards):   0%|          | 0/6185382 [00:00<?, ? examples/s]

In [11]:
eng_tok = AutoTokenizer.from_pretrained('albert-base-v2')

In [13]:
new_tok = eng_tok.train_new_from_iterator(raw_dataset, vocab_size=30_000)

KeyboardInterrupt: 